In [80]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import  VotingClassifier
from sklearn.model_selection import  cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import  r2_score
from sklearn.metrics import  mean_squared_error
from sklearn.model_selection import cross_val_score

In [48]:
def convet(df):
    #convet the categorical data to numeric data
    train_data=df.copy()
    for col in train_data.columns:
        if train_data[col].dtype=='O':
            n=1  #the number replacing the categorical data
            for value in set(train_data[col]):
                train_data.ix[train_data[col]==value,col]=n
                n+=1
    return(train_data)


In [49]:
df_train=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

In [50]:
#prepare the data for training
index=list(df_train.columns)
train_data=df_train[index[:-1]]
train_label=df_train[index[-1]]

In [51]:
#decide to get rid of features that have more than half of missing informations
NAs=pd.concat([train_data.isnull().sum(),test_data.isnull().sum()],axis=1,keys=['train','test'])
#the featrues to be gotted rid of in training data
feature1 =train_data.columns[train_data.isnull().sum()/train_data.shape[0]>0.5]
#the featrues to be gotted rid of in testing data
feature2 =test_data.columns[test_data.isnull().sum()/test_data.shape[0]>0.5]
#the featrues unioning the features removed in training data and those in testing data
features_unioned=set.union(set(feature1),set(feature2),set(['Utilities', 'RoofMatl', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'Heating', 'LowQualFinSF',
               'BsmtFullBath', 'BsmtHalfBath', 'Functional', 'GarageYrBlt', 'GarageArea', 'GarageCond', 'WoodDeckSF',
               'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC', 'Fence', 'MiscFeature', 'MiscVal']))
print(feature1)
print(feature2)
print(features_unioned)
#trainging data and testing data that features having more than half of missing values have been gotten rid of 
train_data=train_data[train_data.columns.difference(list(features_unioned))]
test_data=test_data[test_data.columns.difference(list(features_unioned))]

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')
Index(['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')
{'BsmtHalfBath', 'PoolQC', 'GarageYrBlt', 'MiscVal', 'Functional', 'RoofMatl', 'GarageCond', 'Alley', 'BsmtFullBath', 'BsmtFinSF1', 'WoodDeckSF', 'MiscFeature', 'Utilities', 'LowQualFinSF', 'OpenPorchSF', 'BsmtUnfSF', 'EnclosedPorch', 'PoolArea', 'MasVnrArea', 'GarageArea', 'Fence', 'FireplaceQu', 'Heating', 'BsmtFinSF2', '3SsnPorch', 'ScreenPorch'}


fill the missing values in train data

In [52]:
# MSSubClass as str
features=train_data.copy()
features['MSSubClass'] = features['MSSubClass'].astype(str)

# MSZoning NA in pred. filling with most popular values
features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])

# LotFrontage  NA in all. I suppose NA means 0
features['LotFrontage'] = features['LotFrontage'].fillna(features['LotFrontage'].mean())

# Alley  NA in all. NA means no access
#features['Alley'] = features['Alley'].fillna('NOACCESS')

# Converting OverallCond to str
features.OverallCond = features.OverallCond.astype(str)

# MasVnrType NA in all. filling with most popular values
features['MasVnrType'] = features['MasVnrType'].fillna(features['MasVnrType'].mode()[0])

# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
# NA in all. NA means No basement
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('NoBSMT')

# TotalBsmtSF  NA in pred. I suppose NA means 0
features['TotalBsmtSF'] = features['TotalBsmtSF'].fillna(0)

# Electrical NA in pred. filling with most popular values
features['Electrical'] = features['Electrical'].fillna(features['Electrical'].mode()[0])

# KitchenAbvGr to categorical
features['KitchenAbvGr'] = features['KitchenAbvGr'].astype(str)

# KitchenQual NA in pred. filling with most popular values
features['KitchenQual'] = features['KitchenQual'].fillna(features['KitchenQual'].mode()[0])

# FireplaceQu  NA in all. NA means No Fireplace
#features['FireplaceQu'] = features['FireplaceQu'].fillna('NoFP')

# GarageType, GarageFinish, GarageQual  NA in all. NA means No Garage
for col in ('GarageType', 'GarageFinish', 'GarageQual'):
    features[col] = features[col].fillna('NoGRG')

# GarageCars  NA in pred. I suppose NA means 0
features['GarageCars'] = features['GarageCars'].fillna(0.0)

# SaleType NA in pred. filling with most popular values
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

# Year and Month to categorical
features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

# Adding total sqfootage feature and removing Basement, 1st and 2nd floor features
features['TotalSF'] = features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
features.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
train_data=features

fill the missing value in test data

In [53]:
# MSSubClass as str
features=test_data.copy()
features['MSSubClass'] = features['MSSubClass'].astype(str)

# MSZoning NA in pred. filling with most popular values
features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])

# LotFrontage  NA in all. I suppose NA means 0
features['LotFrontage'] = features['LotFrontage'].fillna(features['LotFrontage'].mean())

# Alley  NA in all. NA means no access
#features['Alley'] = features['Alley'].fillna('NOACCESS')

# Converting OverallCond to str
features.OverallCond = features.OverallCond.astype(str)

# MasVnrType NA in all. filling with most popular values
features['MasVnrType'] = features['MasVnrType'].fillna(features['MasVnrType'].mode()[0])

# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2
# NA in all. NA means No basement
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    features[col] = features[col].fillna('NoBSMT')

# TotalBsmtSF  NA in pred. I suppose NA means 0
features['TotalBsmtSF'] = features['TotalBsmtSF'].fillna(0)

# Electrical NA in pred. filling with most popular values
features['Electrical'] = features['Electrical'].fillna(features['Electrical'].mode()[0])

# KitchenAbvGr to categorical
features['KitchenAbvGr'] = features['KitchenAbvGr'].astype(str)

# KitchenQual NA in pred. filling with most popular values
features['KitchenQual'] = features['KitchenQual'].fillna(features['KitchenQual'].mode()[0])

# FireplaceQu  NA in all. NA means No Fireplace
#features['FireplaceQu'] = features['FireplaceQu'].fillna('NoFP')

# GarageType, GarageFinish, GarageQual  NA in all. NA means No Garage
for col in ('GarageType', 'GarageFinish', 'GarageQual'):
    features[col] = features[col].fillna('NoGRG')

# GarageCars  NA in pred. I suppose NA means 0
features['GarageCars'] = features['GarageCars'].fillna(0.0)

# SaleType NA in pred. filling with most popular values
features['SaleType'] = features['SaleType'].fillna(features['SaleType'].mode()[0])

# Year and Month to categorical
features['YrSold'] = features['YrSold'].astype(str)
features['MoSold'] = features['MoSold'].astype(str)

# Adding total sqfootage feature and removing Basement, 1st and 2nd floor features
features['TotalSF'] = features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
features.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
test_data=features

In [54]:
#logtransforming the scaleprice
train_label=np.log(train_label)
#standardizing numeric value
for col in ['LotFrontage', 'LotArea', 'GrLivArea', 'TotalSF']:
    train_data[col]=(train_data[col]-train_data[col].mean())/train_data[col].std()

conveting the categorical value to numeric data

In [55]:
train_data=convet(df=train_data)
test_data=convet(df=test_data)

In [56]:
#prepare the valid data
valid_data=train_data[:400]
valid_label=train_label[:400]
train_data=train_data[400:]
train_label=train_label[400:]


0.87145756868


In [75]:
#build the RandomForest model
clf_RF=RandomForestRegressor(n_estimators=100,criterion='mae',max_features=0.8,
                          max_depth=6,min_samples_leaf=2,min_samples_split=2,
                          bootstrap=True,oob_score=True,n_jobs=-1,random_state=22)
clf_RF.fit(X=train_data,y=train_label)
print(clf1.score(X=valid_data,y=valid_label))

0.87145756868


In [76]:
#build the AdaBoost
from sklearn.ensemble import  AdaBoostRegressor
from sklearn.tree import  DecisionTreeRegressor
regr=DecisionTreeRegressor(criterion='mse',splitter='best',max_features=None,max_depth=None,min_samples_split=2,
                           random_state=33)
clf_Ada=AdaBoostRegressor(base_estimator=regr,n_estimators=100,learning_rate=0.1,loss='square',random_state=44)
clf_Ada.fit(X=train_data,y=train_label)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=33,
           splitter='best'),
         learning_rate=0.1, loss='square', n_estimators=100,
         random_state=44)

In [72]:
#bulid the GradientBoostedRegressionTree(GBRT)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import  partial_dependence
clf_GBRT=GradientBoostingRegressor(n_estimators=100,loss='ls',learning_rate=0.1,max_depth=4,subsample=0.8,max_features=0.7,
                                  random_state=44)
clf_GBRT.fit(X=valid_data,y=valid_label)
importances=clf_GBRT.feature_importances_
'''
print('oob_improvement_:',clf_GBRT.oob_improvement_)
print('train_score_,',clf_GBRT.train_score_)
for i in clf_GBRT.staged_predict(X=valid_data):
    score=mean_squared_error(valid_label,i)
    print(score)
'''
clf_GBRT.feature_importances_
fig, axs=partial_dependence.plot_partial_dependence(clf_GBRT,train_data,[0,3,4])
plt.show()

In [83]:
#bulid VotingClassification
from sklearn.ensemble import VotingClas
eclf=VotingClassifier(estimators=[('RF',clf_RF),('Ada',clf_Ada),('GBRT',clf_GBRT)],voting='hard',n_jobs=-1)
eclf.fit(X=train_data,y=train_label)


VotingClassifier(estimators=[('RF', RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=6,
           max_features=0.8, max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=2, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_...         presort='auto', random_state=44, subsample=0.8, verbose=0,
             warm_start=False))],
         n_jobs=-1, voting='hard', weights=None)

In [ ]:
eclf.score(X=valid_data,y=valid_label)

In [64]:
importances=clf.feature_importances_
std=np.std([tree.feature_importances_ for tree in clf.estimators_],axis=0)
indices = np.argsort(importances)[::-1]
plt.figure()
plt.title('featrue_importances')
plt.bar(range(train_data.shape[1]),importances[indices],color="r",yerr=std[indices],align="center")
plt.xticks(range(train_data.shape[1]), indices)
plt.xlim([-1, train_data.shape[1]])
plt.show()

In [11]:
def test(clf,test_data,test_label):
    length=test_data.shape[0]
    index=[x for x in range(length)]
    for i in range(10):
        index_=random.sample(index,200)
        prediction=clf.predict(test_data.loc[index_])
        mse=mean_squared_error(prediction,test_label[index_])
        print('the mean squared error :%f'%(mse))
test(clf,valid_data,valid_label)

the mean squared error :0.023710
the mean squared error :0.021413
the mean squared error :0.018689
the mean squared error :0.021803
the mean squared error :0.024927
the mean squared error :0.023364
the mean squared error :0.018464
the mean squared error :0.019267
the mean squared error :0.018481
the mean squared error :0.018793


In [ ]:
from sklearn.ensemble import  BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import  cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import  train_test_split
from sklearn.metrics import r2_score